In [254]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
import pandas as pd
import numpy as np

resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
df_full= yf.download(tickers, start=datetime.datetime(2013,1,2), end=pd.to_datetime("today"))
print(df_full)

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
             Adj Close                                                 \
                     A        AAL         AAP        AAPL        ABBV   
Date                                                                    
2013-01-02   27.765127  13.179525   70.885185   17.065252   24.738327   
2013-01-03   27.864569  12.877850   70.885185   16.849854   24.534056   
2013-01-04   28.414839  13.886582   71.990173   16.380503   24.224121   
2013-01-07   28.209320  13.990282   71.745720   16.284145   24.273428   
2013-01-08   27.983902  14.291961   70.572281   16.327978   23.745136   
...                ...        ...         ...         ...         ...   
2021-01-15  126.209999  15.760000  165.250000  127.139999  110.519997   
2021-01-19  127.349998  15.970000  165.929993  127.830002  112.

In [255]:
df_full = df_full['Adj Close']
df_full

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,27.765127,13.179525,70.885185,17.065252,24.738327,38.232620,13.690000,27.273100,58.187748,38.340000,...,31.153555,64.100830,38.399136,14.614850,24.603680,41.311356,63.162487,40.959999,19.460474,NaN
2013-01-03,27.864569,12.877850,70.885185,16.849854,24.534056,38.153492,13.450000,28.311268,57.977104,37.750000,...,31.094772,63.985226,38.294888,14.614850,24.523157,41.569702,64.067444,41.000000,19.487015,NaN
2013-01-04,28.414839,13.886582,71.990173,16.380503,24.224121,38.408436,13.410000,28.141081,58.297276,38.130001,...,30.657999,64.281471,38.626568,14.452236,24.487370,42.024899,64.393997,40.669998,20.044556,NaN
2013-01-07,28.209320,13.990282,71.745720,16.284145,24.273428,38.531517,13.250000,28.370836,58.044502,37.939999,...,30.531998,63.537216,38.759247,14.696153,24.165289,41.760395,64.552605,40.900002,19.947205,NaN
2013-01-08,27.983902,14.291961,70.572281,16.327978,23.745136,38.478764,13.210000,28.379343,58.381527,38.139999,...,29.910440,63.934666,38.313847,14.675826,23.870043,40.007313,64.645927,40.930000,19.548969,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-15,126.209999,15.760000,165.250000,127.139999,110.519997,106.800003,329.690002,111.300003,253.649994,458.079987,...,136.600006,47.889999,58.439999,22.490000,103.500000,106.620003,160.119995,409.220001,49.139999,161.309998
2021-01-19,127.349998,15.970000,165.929993,127.830002,112.180000,103.500000,336.299988,112.570000,255.839996,456.500000,...,139.360001,48.840000,58.320000,21.090000,104.260002,107.389999,159.929993,405.839996,49.330002,162.570007
2021-01-20,128.110001,16.170000,165.179993,132.029999,112.620003,103.790001,347.869995,112.889999,261.670013,469.730011,...,138.000000,49.529999,58.680000,21.049999,104.000000,109.410004,163.460007,406.549988,48.299999,163.360001


In [256]:
df_full.columns.get_level_values(0).unique()

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE',
       ...
       'XLNX', 'XOM', 'XRAY', 'XRX', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION',
       'ZTS'],
      dtype='object', length=505)

In [359]:
# 2013 starts 0; 2014 starts 252; 2015 starts 252*2; 2016 starts 252*3; 2017 starts 252*4+2; 2018 starts 252*5+2; 
# 2019 starts 252*6+3; 2020 starts 252*7+3; 2021 starts 252*8+4
start = 0 
end = 252*2
df = df_full.iloc[0:252*2,:]
df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,27.765127,13.179525,70.885185,17.065252,24.738327,38.232620,13.690000,27.273100,58.187748,38.340000,...,31.153555,64.100830,38.399136,14.614850,24.603680,41.311356,63.162487,40.959999,19.460474,NaN
2013-01-03,27.864569,12.877850,70.885185,16.849854,24.534056,38.153492,13.450000,28.311268,57.977104,37.750000,...,31.094772,63.985226,38.294888,14.614850,24.523157,41.569702,64.067444,41.000000,19.487015,NaN
2013-01-04,28.414839,13.886582,71.990173,16.380503,24.224121,38.408436,13.410000,28.141081,58.297276,38.130001,...,30.657999,64.281471,38.626568,14.452236,24.487370,42.024899,64.393997,40.669998,20.044556,NaN
2013-01-07,28.209320,13.990282,71.745720,16.284145,24.273428,38.531517,13.250000,28.370836,58.044502,37.939999,...,30.531998,63.537216,38.759247,14.696153,24.165289,41.760395,64.552605,40.900002,19.947205,NaN
2013-01-08,27.983902,14.291961,70.572281,16.327978,23.745136,38.478764,13.210000,28.379343,58.381527,38.139999,...,29.910440,63.934666,38.313847,14.675826,23.870043,40.007313,64.645927,40.930000,19.548969,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-24,38.784866,48.730118,157.396942,25.508888,50.067284,82.871986,38.459999,40.408478,80.830475,74.739998,...,38.883419,71.588036,51.525497,29.763666,35.763321,46.623360,108.686729,78.330002,25.497263,42.021610
2014-12-26,39.020618,49.232281,157.171082,25.959806,50.649563,82.790573,38.639999,40.541119,80.777367,74.669998,...,38.733490,71.152901,51.602169,29.890501,35.938633,46.847538,108.743881,78.099998,25.533026,42.366680
2014-12-29,38.973473,50.075562,158.369476,25.941589,50.770542,82.899139,38.480000,40.320057,80.122383,74.129997,...,38.618847,71.046028,51.496742,30.024273,35.855583,47.116558,108.105743,78.050003,25.676064,42.184563


In [277]:
# drop tickers that didn't contain full data in the time range
df=df.drop([col for col in df.columns if np.isnan(df[col][0])==1], axis=1)
df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,27.765127,13.179525,70.885185,17.065252,24.738327,38.232620,13.690000,27.273100,58.187748,38.340000,...,20.801807,31.153555,64.100830,38.399136,14.614850,24.603680,41.311356,63.162487,40.959999,19.460474
2013-01-03,27.864569,12.877850,70.885185,16.849854,24.534056,38.153492,13.450000,28.311268,57.977104,37.750000,...,20.741024,31.094772,63.985226,38.294888,14.614850,24.523157,41.569702,64.067444,41.000000,19.487015
2013-01-04,28.414839,13.886582,71.990173,16.380503,24.224121,38.408436,13.410000,28.141081,58.297276,38.130001,...,20.817001,30.657999,64.281471,38.626568,14.452236,24.487370,42.024899,64.393997,40.669998,20.044556
2013-01-07,28.209320,13.990282,71.745720,16.284145,24.273428,38.531517,13.250000,28.370836,58.044502,37.939999,...,20.596676,30.531998,63.537216,38.759247,14.696153,24.165289,41.760395,64.552605,40.900002,19.947205
2013-01-08,27.983902,14.291961,70.572281,16.327978,23.745136,38.478764,13.210000,28.379343,58.381527,38.139999,...,20.634663,29.910440,63.934666,38.313847,14.675826,23.870043,40.007313,64.645927,40.930000,19.548969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-12-24,38.784866,48.730118,157.396942,25.508888,50.067284,82.871986,38.459999,40.408478,80.830475,74.739998,...,30.021887,38.883419,71.588036,51.525497,29.763666,35.763321,46.623360,108.686729,78.330002,25.497263
2014-12-26,39.020618,49.232281,157.171082,25.959806,50.649563,82.790573,38.639999,40.541119,80.777367,74.669998,...,30.311930,38.733490,71.152901,51.602169,29.890501,35.938633,46.847538,108.743881,78.099998,25.533026
2014-12-29,38.973473,50.075562,158.369476,25.941589,50.770542,82.899139,38.480000,40.320057,80.122383,74.129997,...,30.867104,38.618847,71.046028,51.496742,30.024273,35.855583,47.116558,108.105743,78.050003,25.676064


In [278]:
# calculate monthly returns 
stock_monthly_returns = df.resample('M').ffill().pct_change()
stock_monthly_returns

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-02-28,-0.073694,-0.059524,0.038357,-0.025286,0.006269,0.045008,0.149821,-0.002656,0.034358,0.039122,...,0.033118,0.027459,0.001690,-0.008142,0.012485,-0.011367,0.008316,0.004826,0.033272,0.036015
2013-03-31,0.014706,0.263589,0.083468,0.002854,0.104550,0.090043,0.163965,0.045280,0.021651,0.107097,...,0.044603,0.024148,0.006253,0.026177,0.067695,0.002181,0.098656,0.006228,0.053891,0.034783
2013-04-30,-0.012628,-0.004125,0.014882,0.000271,0.139681,0.051895,-0.010712,0.049259,0.083518,0.035846,...,0.070370,-0.006812,-0.012429,-0.002121,-0.002325,0.006894,-0.048327,0.016352,-0.010185,-0.014806
2013-05-31,0.096766,0.039645,-0.028135,0.022419,-0.072964,0.003090,0.167840,-0.006771,0.008227,-0.048137,...,-0.096572,0.079274,0.023664,-0.013932,0.024476,0.018137,-0.005431,0.026945,-0.021222,0.141779
2013-06-30,-0.056558,-0.065453,-0.003579,-0.118293,-0.031623,0.032360,-0.000464,-0.048814,-0.123615,0.061757,...,-0.003769,-0.025584,-0.001326,-0.017619,0.038569,-0.042644,0.023469,-0.042919,-0.048620,0.030281
2013-07-31,0.046071,0.178441,0.016262,0.141225,0.110259,0.043704,0.163265,0.054422,0.025709,0.037752,...,0.056810,0.178743,0.037631,0.046875,0.069460,-0.074610,0.056478,0.113958,0.062845,0.024896
2013-08-31,0.042701,-0.164858,-0.029337,0.083767,-0.063105,-0.019620,-0.061005,-0.090090,-0.021135,-0.032360,...,-0.067780,-0.065011,-0.063867,-0.020756,0.028866,-0.001374,-0.039770,-0.052588,-0.012346,-0.055026
2013-09-30,0.101379,0.173267,0.033372,-0.021489,0.049754,0.073436,-0.190658,-0.004200,0.019238,0.135301,...,-0.000893,0.078996,-0.012850,0.035303,0.036833,0.127118,0.019566,0.041048,-0.000877,-0.019664


In [279]:
stock_monthly_returns.mean()

A       0.012599
AAL     0.066282
AAP     0.036578
AAPL    0.027459
ABBV    0.030080
          ...   
XYL     0.017683
YUM     0.008688
ZBH     0.020094
ZBRA    0.027625
ZION    0.010163
Length: 467, dtype: float64

In [198]:
# from sklearn import preprocessing
# min_max_scaler = preprocessing.MinMaxScaler()
# normalized_stock_monthly_returns = min_max_scaler.fit_transform(pd.DataFrame(stock_monthly_returns.mean()))

In [280]:
# normalized_stock_monthly_returns

In [281]:
# the higher the rank, the more returns
stock_monthly_returns.mean().rank(ascending=False)

A       343.0
AAL       5.0
AAP      42.0
AAPL     99.0
ABBV     77.0
        ...  
XYL     239.0
YUM     402.0
ZBH     201.0
ZBRA     98.0
ZION    378.0
Length: 467, dtype: float64

In [282]:
stock_monthly_returns.std()

A       0.048683
AAL     0.123282
AAP     0.069195
AAPL    0.067537
ABBV    0.064719
          ...   
XYL     0.078857
YUM     0.066712
ZBH     0.041460
ZBRA    0.067730
ZION    0.045120
Length: 467, dtype: float64

In [284]:
# normalized_stock_monthly_returns_std = min_max_scaler.fit_transform(pd.DataFrame(-1*stock_monthly_returns.std()))
# normalized_stock_monthly_returns_std

In [285]:
# indexes = pd.DataFrame(normalized_stock_monthly_returns+normalized_stock_monthly_returns_std,columns = ['measure']).nlargest(10,'measure').index
# indexes

In [286]:
# top 10 tickers using normalizing method
# stock_monthly_returns.mean()[indexes].index

In [287]:
# the higher the rank, the less volatile
stock_monthly_returns.std().rank(ascending=True)

A       175.0
AAL     460.0
AAP     367.0
AAPL    356.0
ABBV    338.0
        ...  
XYL     407.0
YUM     348.0
ZBH      76.0
ZBRA    360.0
ZION    125.0
Length: 467, dtype: float64

In [290]:
tickers = (stock_monthly_returns.mean().rank(ascending=False)+stock_monthly_returns.std().rank(ascending=True)).nsmallest(10).reset_index()['index']
tickers = tickers.tolist()

In [291]:
# adding walmart here because I feel like walmart stock would go up
# tickers.append('WMT')
# tickers

In [292]:
df=df[tickers]
df

,LMT,NOC,UNP,CTAS,GD,MAR,WAB,MMC,FIS,WFC
Date,,,,,,,,,,
2013-01-02,73.214172,58.875595,52.704723,37.802883,58.949398,35.137585,42.766594,29.893679,31.965376,27.292976
2013-01-03,73.433952,58.676979,52.778572,37.973927,59.181168,35.418396,43.424911,29.868435,31.742289,27.067162
2013-01-04,73.708710,59.385166,53.701706,38.343044,58.833492,35.536152,43.280758,29.784264,32.108173,27.207323
2013-01-07,73.959900,59.307430,53.447338,38.262024,59.487469,35.336861,42.877117,29.716944,31.947533,27.074944
2013-01-08,72.970810,58.158752,53.410400,38.370056,58.287121,35.237225,42.829060,29.355053,31.840448,27.028219
...,...,...,...,...,...,...,...,...,...,...
2014-12-24,167.452347,137.711395,105.112129,74.712440,122.443054,72.934967,85.376472,51.406593,58.901764,45.588005
2014-12-26,167.801865,137.520859,105.190758,74.675026,122.313309,73.251480,85.782257,51.177418,58.799927,45.538582
2014-12-29,166.778915,137.094360,105.461624,74.441147,121.690392,73.614525,85.994804,51.388966,58.846218,45.892807


In [293]:
month_returns = df.resample('M').ffill().pct_change().mean()
month_returns

LMT     0.039416
NOC     0.039080
UNP     0.029405
CTAS    0.029487
GD      0.034968
MAR     0.031424
WAB     0.028231
MMC     0.023170
FIS     0.024990
WFC     0.022494
dtype: float64

In [294]:
quarter_returns = df.resample('Q').ffill().pct_change().mean()
quarter_returns

LMT     0.116578
NOC     0.120874
UNP     0.084865
CTAS    0.093203
GD      0.107414
MAR     0.097627
WAB     0.081671
MMC     0.067033
FIS     0.072513
WFC     0.066287
dtype: float64

In [295]:
half_year_returns = df.resample('6M').ffill().pct_change().mean()
half_year_returns

LMT     0.250433
NOC     0.248848
UNP     0.179600
CTAS    0.180818
GD      0.214787
MAR     0.194118
WAB     0.171961
MMC     0.138754
FIS     0.148789
WFC     0.136420
dtype: float64

In [296]:
year_returns = df.resample('Y').ffill().pct_change().mean()
year_returns

LMT     0.347909
NOC     0.313891
UNP     0.451952
CTAS    0.348425
GD      0.472599
MAR     0.600433
WAB     0.172828
MMC     0.209348
FIS     0.178513
WFC     0.241081
dtype: float64

In [297]:
returns = pd.concat([month_returns,quarter_returns, half_year_returns,year_returns], axis=1) # Creating a table for visualising returns and volatility of assets
returns.columns = ['month_returns','quarter_returns', 'half_year_returns', 'year_returns']

In [298]:
returns = returns.sort_values(by=['year_returns','half_year_returns','quarter_returns','month_returns'],ascending=False)
print(returns.to_string())

      month_returns  quarter_returns  half_year_returns  year_returns
MAR        0.031424         0.097627           0.194118      0.600433
GD         0.034968         0.107414           0.214787      0.472599
UNP        0.029405         0.084865           0.179600      0.451952
CTAS       0.029487         0.093203           0.180818      0.348425
LMT        0.039416         0.116578           0.250433      0.347909
NOC        0.039080         0.120874           0.248848      0.313891
WFC        0.022494         0.066287           0.136420      0.241081
MMC        0.023170         0.067033           0.138754      0.209348
FIS        0.024990         0.072513           0.148789      0.178513
WAB        0.028231         0.081671           0.171961      0.172828


In [299]:
# calculate (max-min)/mean as a way to measure volatility
(df.resample('Y').max()-df.resample('Y').min())/df.resample('Y').mean()


,LMT,NOC,UNP,CTAS,GD,MAR,WAB,MMC,FIS,WFC
Date,,,,,,,,,,
2013-12-31,0.593916,0.617964,0.300934,0.386104,0.411156,0.272305,0.529297,0.354833,0.433842,0.297752
2014-12-31,0.341098,0.351935,0.420612,0.405386,0.448942,0.511151,0.256943,0.292922,0.290230,0.250984


In [300]:
# Yearly returns for individual companies
ind_er = df.resample('Y').last().pct_change().mean()
ind_er

LMT     0.347909
NOC     0.313891
UNP     0.451952
CTAS    0.348425
GD      0.472599
MAR     0.600433
WAB     0.172828
MMC     0.209348
FIS     0.178513
WFC     0.241081
dtype: float64

In [303]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [304]:
# Calculate annualised mean (daily) historical return from input (daily) asset prices. By default, this uses the arithmetic mean (correct if log_returns are used).
mu = expected_returns.mean_historical_return(df,compounding =False)
mu

LMT     0.418891
NOC     0.426894
UNP     0.357887
CTAS    0.345978
GD      0.368689
MAR     0.379780
WAB     0.358214
MMC     0.271900
FIS     0.309512
WFC     0.263440
dtype: float64

In [305]:
expected_returns.ema_historical_return(df, returns_data=False, compounding=False, span=500, frequency=252)

LMT     0.388470
NOC     0.402180
UNP     0.366045
CTAS    0.402434
GD      0.370430
MAR     0.432960
WAB     0.287956
MMC     0.259935
FIS     0.280930
WFC     0.240116
Name: 2014-12-31 00:00:00, dtype: float64

In [306]:
S = risk_models.sample_cov(df)
S

,LMT,NOC,UNP,CTAS,GD,MAR,WAB,MMC,FIS,WFC
LMT,0.028130,0.022195,0.012382,0.011854,0.019917,0.012897,0.014400,0.011112,0.013510,0.010926
NOC,0.022195,0.031098,0.014790,0.013824,0.022122,0.014628,0.017767,0.012828,0.015202,0.012889
UNP,0.012382,0.014790,0.033515,0.014573,0.016344,0.015347,0.021838,0.013016,0.014842,0.014269
CTAS,0.011854,0.013824,0.014573,0.027125,0.013098,0.013233,0.016380,0.011116,0.012699,0.011515
GD,0.019917,0.022122,0.016344,0.013098,0.032809,0.015771,0.018746,0.012860,0.016041,0.013003
MAR,0.012897,0.014628,0.015347,0.013233,0.015771,0.031453,0.018931,0.013712,0.015158,0.012807
WAB,0.014400,0.017767,0.021838,0.016380,0.018746,0.018931,0.040331,0.016320,0.017449,0.015604
MMC,0.011112,0.012828,0.013016,0.011116,0.012860,0.013712,0.016320,0.019105,0.013988,0.012721
FIS,0.013510,0.015202,0.014842,0.012699,0.016041,0.015158,0.017449,0.013988,0.028965,0.013400
WFC,0.010926,0.012889,0.014269,0.011515,0.013003,0.012807,0.015604,0.012721,0.013400,0.022108


In [307]:
risk_models.CovarianceShrinkage(df).ledoit_wolf(shrinkage_target='constant_variance')

,LMT,NOC,UNP,CTAS,GD,MAR,WAB,MMC,FIS,WFC
LMT,0.028095,0.021800,0.012161,0.011643,0.019563,0.012668,0.014144,0.010914,0.013270,0.010732
NOC,0.021800,0.031011,0.014527,0.013579,0.021729,0.014367,0.017452,0.012600,0.014932,0.012660
UNP,0.012161,0.014527,0.033385,0.014314,0.016053,0.015074,0.021449,0.012785,0.014578,0.014015
CTAS,0.011643,0.013579,0.014314,0.027108,0.012865,0.012998,0.016088,0.010918,0.012473,0.011311
GD,0.019563,0.021729,0.016053,0.012865,0.032691,0.015490,0.018412,0.012631,0.015755,0.012771
MAR,0.012668,0.014367,0.015074,0.012998,0.015490,0.031359,0.018594,0.013468,0.014889,0.012579
WAB,0.014144,0.017452,0.021449,0.016088,0.018412,0.018594,0.040079,0.016030,0.017139,0.015326
MMC,0.010914,0.012600,0.012785,0.010918,0.012631,0.013468,0.016030,0.019230,0.013739,0.012495
FIS,0.013270,0.014932,0.014578,0.012473,0.015755,0.014889,0.017139,0.013739,0.028916,0.013162
WFC,0.010732,0.012660,0.014015,0.011311,0.012771,0.012579,0.015326,0.012495,0.013162,0.022180


In [308]:
risk_models.exp_cov(df)

,LMT,NOC,UNP,CTAS,GD,MAR,WAB,MMC,FIS,WFC
LMT,0.027905,0.024848,0.014906,0.014118,0.021805,0.012748,0.015573,0.010507,0.014429,0.010924
NOC,0.024848,0.042277,0.024221,0.019430,0.026600,0.018156,0.024579,0.015092,0.017928,0.015646
UNP,0.014906,0.024221,0.050226,0.021378,0.022028,0.016255,0.033867,0.015526,0.019748,0.018312
CTAS,0.014118,0.019430,0.021378,0.041576,0.016850,0.014904,0.022687,0.012324,0.014136,0.012665
GD,0.021805,0.026600,0.022028,0.016850,0.031842,0.017476,0.023675,0.014193,0.018275,0.015467
MAR,0.012748,0.018156,0.016255,0.014904,0.017476,0.030773,0.021261,0.012514,0.014856,0.013383
WAB,0.015573,0.024579,0.033867,0.022687,0.023675,0.021261,0.049068,0.017700,0.019987,0.019882
MMC,0.010507,0.015092,0.015526,0.012324,0.014193,0.012514,0.017700,0.017304,0.013732,0.013057
FIS,0.014429,0.017928,0.019748,0.014136,0.018275,0.014856,0.019987,0.013732,0.024282,0.016119
WFC,0.010924,0.015646,0.018312,0.012665,0.015467,0.013383,0.019882,0.013057,0.016119,0.021426


In [309]:
ef = EfficientFrontier(mu,S)
ef

In [310]:
weights = ef.max_sharpe()
weights

OrderedDict([('LMT', 0.335368054124722),
             ('NOC', 0.1533884223178931),
             ('UNP', 0.1167488811518034),
             ('CTAS', 0.1907831805972853),
             ('GD', 0.0),
             ('MAR', 0.2037114618082964),
             ('WAB', 0.0),
             ('MMC', 0.0),
             ('FIS', 0.0),
             ('WFC', 0.0)])

In [311]:
cleaned_weights = ef.clean_weights()
cleaned_weights

OrderedDict([('LMT', 0.33537),
             ('NOC', 0.15339),
             ('UNP', 0.11675),
             ('CTAS', 0.19078),
             ('GD', 0.0),
             ('MAR', 0.20371),
             ('WAB', 0.0),
             ('MMC', 0.0),
             ('FIS', 0.0),
             ('WFC', 0.0)])

In [312]:
ef.portfolio_performance(verbose =True)

Expected annual return: 39.1%
Annual volatility: 13.4%
Sharpe Ratio: 2.78


(0.3911187451462126, 0.13355826435474374, 2.7787029648759516)

In [313]:
from pypfopt.discrete_allocation import DiscreteAllocation,get_latest_prices

In [314]:
portfolio_val = 10000
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights,latest_prices,total_portfolio_value =portfolio_val)
allocation,leftover = da.lp_portfolio()
print("Discrete Allocaiton: ", allocation)
print("Funds Remaining: ", leftover)

Discrete Allocaiton:  {'LMT': 20, 'NOC': 12, 'UNP': 11, 'CTAS': 26, 'MAR': 28}
Funds Remaining:  25.002708435058594


# Method 2 - Optimization from 500 tickers

In [88]:
import bs4 as bs
import requests
import yfinance as yf
import datetime

resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

tickers = [s.replace('\n', '') for s in tickers]
start = datetime.datetime(2017,1,3)
# end = pd.to_datetime("today")
end = datetime.datetime(2018,12,31)
df= yf.download(tickers, start=start, end=end)
print(df)

[*********************100%***********************]  505 of 505 completed

11 Failed downloads:
- CTVA: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- FOX: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- FOXA: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- VIAC: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- LUMN: No data found for this date range, symbol may be delisted
- CARR: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- OTIS: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- DOW: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
- VNT: Data doesn't exist for startDate = 1483419600, endDate = 1546232400
            Adj Close                                                          \
                    A        AAL        

In [89]:
df = df['Adj Close']
df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,44.900917,44.741226,168.094315,27.500973,50.628330,76.909164,112.360001,36.148453,108.617332,103.480003,...,55.070808,74.296684,56.778732,23.527445,47.210743,58.928459,100.094238,86.250000,39.293274,52.175865
2017-01-04,45.490070,45.127758,169.473709,27.470192,51.342216,78.817696,115.739998,36.435421,108.878471,104.139999,...,54.669926,73.479240,57.243649,24.415272,47.914391,59.142868,101.014511,87.029999,39.857468,52.682140
2017-01-05,44.949219,44.345032,169.355484,27.609884,51.731594,77.905327,114.809998,36.750153,107.246315,105.910004,...,54.007980,72.383888,56.497826,24.312834,47.476990,59.338650,101.663528,84.750000,39.211372,52.506889
2017-01-06,46.349663,44.654255,167.138504,27.917688,51.747826,78.948051,115.419998,37.749912,108.468102,108.300003,...,55.052166,72.343018,56.459095,23.937212,47.144192,60.056496,101.673210,85.959999,39.466175,52.672401
2017-01-09,46.494530,45.494972,167.039993,28.173399,52.088531,79.581116,117.110001,37.712887,107.255646,108.570000,...,55.061493,71.149574,56.633450,23.800617,46.944500,60.224304,103.649330,85.970001,39.038479,52.526363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,61.487480,29.247076,147.293564,35.854492,73.523094,68.204529,281.079987,63.234695,129.476700,205.160004,...,76.997826,57.892380,34.799927,17.173914,59.819813,83.974777,95.530594,142.899994,36.100338,78.208008
2018-12-26,64.303337,31.776180,153.336807,38.379425,77.786316,70.402199,307.440002,67.150696,134.649185,222.949997,...,82.777306,60.658413,36.497726,17.881933,62.636349,86.840851,100.236870,152.520004,37.975193,81.798798
2018-12-27,65.225601,31.530161,153.000565,38.130344,78.546364,70.874496,315.670013,68.124878,136.005280,225.139999,...,81.883659,60.923531,36.515434,17.863775,63.688908,87.776894,100.167938,155.350006,37.928326,83.189735


In [90]:
# drop tickers that didn't contain full data in the time range
df=df.drop([col for col in df.columns if np.isnan(df[col][0])==1], axis=1)
df

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2017-01-03,44.900917,44.741226,168.094315,27.500973,50.628330,76.909164,112.360001,36.148453,108.617332,103.480003,...,55.070808,74.296684,56.778732,23.527445,47.210743,58.928459,100.094238,86.250000,39.293274,52.175865
2017-01-04,45.490070,45.127758,169.473709,27.470192,51.342216,78.817696,115.739998,36.435421,108.878471,104.139999,...,54.669926,73.479240,57.243649,24.415272,47.914391,59.142868,101.014511,87.029999,39.857468,52.682140
2017-01-05,44.949219,44.345032,169.355484,27.609884,51.731594,77.905327,114.809998,36.750153,107.246315,105.910004,...,54.007980,72.383888,56.497826,24.312834,47.476990,59.338650,101.663528,84.750000,39.211372,52.506889
2017-01-06,46.349663,44.654255,167.138504,27.917688,51.747826,78.948051,115.419998,37.749912,108.468102,108.300003,...,55.052166,72.343018,56.459095,23.937212,47.144192,60.056496,101.673210,85.959999,39.466175,52.672401
2017-01-09,46.494530,45.494972,167.039993,28.173399,52.088531,79.581116,117.110001,37.712887,107.255646,108.570000,...,55.061493,71.149574,56.633450,23.800617,46.944500,60.224304,103.649330,85.970001,39.038479,52.526363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,61.487480,29.247076,147.293564,35.854492,73.523094,68.204529,281.079987,63.234695,129.476700,205.160004,...,76.997826,57.892380,34.799927,17.173914,59.819813,83.974777,95.530594,142.899994,36.100338,78.208008
2018-12-26,64.303337,31.776180,153.336807,38.379425,77.786316,70.402199,307.440002,67.150696,134.649185,222.949997,...,82.777306,60.658413,36.497726,17.881933,62.636349,86.840851,100.236870,152.520004,37.975193,81.798798
2018-12-27,65.225601,31.530161,153.000565,38.130344,78.546364,70.874496,315.670013,68.124878,136.005280,225.139999,...,81.883659,60.923531,36.515434,17.863775,63.688908,87.776894,100.167938,155.350006,37.928326,83.189735


In [91]:
mu = expected_returns.mean_historical_return(df)
mu

A       0.202022
AAL    -0.163278
AAP    -0.043581
AAPL    0.177803
ABBV    0.253918
          ...   
YUM     0.224824
ZBH     0.004980
ZBRA    0.344748
ZION   -0.017887
ZTS     0.263897
Length: 493, dtype: float64

In [92]:
S = risk_models.sample_cov(df)
S

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION,ZTS
A,0.049811,0.027513,0.011406,0.022310,0.022396,0.016630,0.034892,0.023244,0.022428,0.036074,...,0.028770,0.013241,0.013058,0.017734,0.023865,0.014482,0.020075,0.025995,0.016513,0.023384
AAL,0.027513,0.120693,0.022311,0.025284,0.020761,0.025352,0.024531,0.023010,0.021421,0.034520,...,0.041727,0.011163,0.019009,0.031500,0.027275,0.016029,0.021016,0.039742,0.032507,0.024055
AAP,0.011406,0.022311,0.129451,0.009497,0.014677,0.018457,0.019310,0.014053,0.015670,0.011318,...,0.016392,0.011248,0.008132,0.018944,0.012525,0.010350,0.012074,0.012670,0.023567,0.015404
AAPL,0.022310,0.025284,0.009497,0.056605,0.017557,0.015477,0.038656,0.020460,0.022622,0.039020,...,0.036561,0.014201,0.014134,0.016441,0.017353,0.012850,0.018215,0.029090,0.016896,0.019337
ABBV,0.022396,0.020761,0.014677,0.017557,0.076965,0.025637,0.030235,0.024233,0.021816,0.026712,...,0.024458,0.014743,0.020294,0.018277,0.019966,0.011362,0.020729,0.017608,0.017954,0.023565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,0.014482,0.016029,0.010350,0.012850,0.011362,0.006949,0.021108,0.013698,0.013219,0.022166,...,0.016992,0.009329,0.005695,0.014655,0.013714,0.028209,0.009209,0.017259,0.011506,0.015507
ZBH,0.020075,0.021016,0.012074,0.018215,0.020729,0.020100,0.025750,0.020890,0.017403,0.025332,...,0.018698,0.010224,0.016992,0.017153,0.016574,0.009209,0.046275,0.015917,0.016532,0.015725
ZBRA,0.025995,0.039742,0.012670,0.029090,0.017608,0.016527,0.046145,0.020822,0.022680,0.040441,...,0.040001,0.017312,-0.008524,0.033390,0.026118,0.017259,0.015917,0.110494,0.020433,0.023723
ZION,0.016513,0.032507,0.023567,0.016896,0.017954,0.016329,0.023744,0.015740,0.017486,0.017049,...,0.022669,0.012713,0.011643,0.021005,0.018733,0.011506,0.016532,0.020433,0.056480,0.015219


In [93]:
ef = EfficientFrontier(mu,S)
ef

In [94]:
weights = ef.efficient_risk(0.2)
weights

OrderedDict([('A', 1.5309e-11),
             ('AAL', 5.0947e-12),
             ('AAP', 8.4619e-12),
             ('AAPL', 1.30313e-11),
             ('ABBV', 2.93916e-11),
             ('ABC', 8.6464e-12),
             ('ABMD', 0.0885390376060024),
             ('ABT', 7.89223e-11),
             ('ACN', 1.22679e-11),
             ('ADBE', 3.41366e-11),
             ('ADI', 1.21504e-11),
             ('ADM', 9.403e-12),
             ('ADP', 1.36147e-11),
             ('ADSK', 1.09153e-11),
             ('AEE', 2.68157e-11),
             ('AEP', 2.95641e-11),
             ('AES', 1.82346e-11),
             ('AFL', 2.30127e-11),
             ('AIG', 6.6512e-12),
             ('AIZ', 1.08347e-11),
             ('AJG', 2.78255e-11),
             ('AKAM', 6.2953e-12),
             ('ALB', 7.2601e-12),
             ('ALGN', 2.64689e-11),
             ('ALK', 6.0466e-12),
             ('ALL', 1.4499e-11),
             ('ALLE', 1.39393e-11),
             ('ALXN', 5.4743e-12),
             ('AMA

In [95]:
cleaned_weights = ef.clean_weights()
from collections import OrderedDict
OrderedDict(sorted(cleaned_weights.items(), key=lambda t: t[1],reverse=True))

OrderedDict([('LW', 0.30447),
             ('NRG', 0.27787),
             ('ETSY', 0.15346),
             ('ABMD', 0.08854),
             ('BA', 0.07988),
             ('ENPH', 0.05968),
             ('CME', 0.01688),
             ('PAYC', 0.01269),
             ('ANTM', 0.00338),
             ('FTNT', 0.00315),
             ('A', 0.0),
             ('AAL', 0.0),
             ('AAP', 0.0),
             ('AAPL', 0.0),
             ('ABBV', 0.0),
             ('ABC', 0.0),
             ('ABT', 0.0),
             ('ACN', 0.0),
             ('ADBE', 0.0),
             ('ADI', 0.0),
             ('ADM', 0.0),
             ('ADP', 0.0),
             ('ADSK', 0.0),
             ('AEE', 0.0),
             ('AEP', 0.0),
             ('AES', 0.0),
             ('AFL', 0.0),
             ('AIG', 0.0),
             ('AIZ', 0.0),
             ('AJG', 0.0),
             ('AKAM', 0.0),
             ('ALB', 0.0),
             ('ALGN', 0.0),
             ('ALK', 0.0),
             ('ALL', 0.0),
       

In [96]:
ef.portfolio_performance(verbose =True)

Expected annual return: 67.9%
Annual volatility: 20.0%
Sharpe Ratio: 3.30


(0.6791486473563748, 0.19999999998047763, 3.2957432371035775)

In [97]:
portfolio_val = 10000
latest_prices = get_latest_prices(df)
weights = cleaned_weights
da = DiscreteAllocation(weights,latest_prices,total_portfolio_value =portfolio_val)
allocation,leftover = da.lp_portfolio()
print("Discrete Allocaiton: ", allocation)
print("Funds Remaining: ", leftover)

Discrete Allocaiton:  {'ABMD': 3, 'BA': 3, 'CME': 1, 'ENPH': 123, 'ETSY': 31, 'LW': 42, 'NRG': 74, 'PAYC': 1}
Funds Remaining:  0.0016536712646484375


# Back Testing

In [315]:
allocation

{'LMT': 20, 'NOC': 12, 'UNP': 11, 'CTAS': 26, 'MAR': 28}

In [328]:
start

0

In [329]:
end

504

In [318]:
latest_prices

LMT     164.161774
NOC     133.745926
UNP     104.089821
CTAS     73.383995
GD      119.051941
MAR      72.637100
WAB      83.946617
MMC      50.454624
FIS      57.587086
WFC      45.159653
Name: 2014-12-31 00:00:00, dtype: float64

In [319]:
initial_value_portfolio = 0
for key in allocation:
    initial_value_portfolio += latest_prices[key]*allocation[key] 
initial_value_portfolio

9974.997291564941

In [331]:
end_value_portfolio = 0 
for key in allocation:
    end_value_portfolio+=allocation[key]*df_full[key][end+252]
end_value_portfolio

10499.156059265137

In [332]:
actual_return = (end_value_portfolio-initial_value_portfolio)/initial_value_portfolio
actual_return

0.05254725914997837

In [333]:
# expected return 
ef.portfolio_performance()[0]

0.3911187451462126

In [334]:
start

0

In [335]:
end

504

In [363]:
result = pd.DataFrame(columns=['start','end','expected_return','actual_return'])
result

,start,end,expected_return,actual_return


In [354]:
df_full.index

DatetimeIndex(['2013-01-02', '2013-01-03', '2013-01-04', '2013-01-07',
               '2013-01-08', '2013-01-09', '2013-01-10', '2013-01-11',
               '2013-01-14', '2013-01-15',
               ...
               '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13',
               '2021-01-14', '2021-01-15', '2021-01-19', '2021-01-20',
               '2021-01-21', '2021-01-22'],
              dtype='datetime64[ns]', name='Date', length=2034, freq=None)

In [360]:
start_date = df_full.index[start]
start_date

Timestamp('2013-01-02 00:00:00')

In [361]:
end_date = df_full.index[end]
end_date

Timestamp('2015-01-02 00:00:00')

In [362]:
result.append({'start':start_date,'end':end_date,'expected_return':ef.portfolio_performance()[0],'actual_return':actual_return},ignore_index=True)

,start,end,expected_return,actual_return
0,2013-01-02,2015-01-02,0.391119,0.052547


In [ ]:
result = pd.DataFrame(columns=['start','end','portfolio','expected_return','actual_return'])

In [483]:
# test multiple years
for start in range(600,252*7+3):
    end = start + 252*2
    if end + 253 <= len(df_full):
        df = df_full.iloc[start:end,:]
        df = df.dropna(how='all')
        df =df.drop([col for col in df.columns if np.isnan(df[col][0])==1], axis=1)
        
#         stock_monthly_returns = df.resample('M').ffill().pct_change()
        # pick top 10 stocks
#         tickers = (stock_monthly_returns.mean().rank(ascending=False)+stock_monthly_returns.std().rank(ascending=True)).nsmallest(15).reset_index()['index']
#         tickers = tickers.tolist()
#         df=df[tickers]
        # Calculate annualised mean (daily) historical return from input (daily) asset prices. By default, this uses the arithmetic mean (correct if log_returns are used).
        mu = expected_returns.mean_historical_return(df,compounding =False)
        S = risk_models.sample_cov(df)
        try:
            ef = EfficientFrontier(mu,S)
            weights = ef.max_sharpe()
            cleaned_weights = ef.clean_weights()
            portfolio_val = 10000
            latest_prices = get_latest_prices(df)
            weights = cleaned_weights
            da = DiscreteAllocation(weights,latest_prices,total_portfolio_value =portfolio_val)
            allocation,leftover = da.lp_portfolio()

            initial_value_portfolio = 0
            for key in allocation:
                initial_value_portfolio += latest_prices[key]*allocation[key] 
            end_value_portfolio = 0 

            for key in allocation:
                end_value_portfolio+=allocation[key]*df_full[key][end+252]
            actual_return = (end_value_portfolio-initial_value_portfolio)/initial_value_portfolio
            start_date = df_full.index[start]
            end_date = df_full.index[end]
            result = result.append({'start':start_date,'end':end_date,'portfolio':list(allocation.items()),'expected_return':ef.portfolio_performance()[0],'actual_return':actual_return},ignore_index=True)
        except Exception:
            pass
    else:
        break
result

,start,end,portfolio,expected_return,actual_return
0,2013-01-02,2015-01-02,"[(ABC, 10), (ABMD, 1), (AMCR, 124), (BBY, 4), ...",0.534718,0.114611
1,2013-01-03,2015-01-05,"[(ABC, 10), (ABMD, 1), (AMCR, 123), (BBY, 3), ...",0.523858,0.123409
2,2013-01-04,2015-01-06,"[(ABC, 11), (ABMD, 2), (AMCR, 125), (BBY, 3), ...",0.514343,0.149813
3,2013-01-07,2015-01-07,"[(ABC, 11), (ABMD, 2), (AMCR, 112), (BBY, 3), ...",0.506118,0.145699
4,2013-01-08,2015-01-08,"[(ABC, 10), (ABMD, 2), (AMCR, 108), (AMD, 1), ...",0.512644,0.130540
5,2013-01-09,2015-01-09,"[(ABC, 10), (ABMD, 2), (AMCR, 107), (BBY, 3), ...",0.523753,0.119110
6,2013-01-10,2015-01-12,"[(ABC, 12), (ABMD, 3), (AMCR, 110), (BBY, 2), ...",0.513385,0.136310
7,2013-01-11,2015-01-13,"[(ABC, 10), (ABMD, 5), (AMCR, 113), (CBOE, 1),...",0.508808,0.121721
8,2013-01-14,2015-01-14,"[(ABC, 10), (ABMD, 4), (AMCR, 115), (AMD, 5), ...",0.506996,0.120682
9,2013-01-16,2015-01-16,"[(ABC, 9), (ABMD, 3), (AMCR, 100), (CNC, 5), (...",0.484019,NaN


In [486]:
pd.set_option('display.max_rows', None)
result[result['actual_return']<0]

,start,end,portfolio,expected_return,actual_return
16,2013-02-25,2015-02-25,"[(AAPL, 19), (ABC, 12), (ABMD, 5), (AMCR, 84),...",0.526714,-0.015955
23,2013-03-19,2015-03-19,"[(AAPL, 14), (ABC, 13), (ABMD, 4), (AMCR, 122)...",0.515751,-0.026236
24,2013-03-20,2015-03-20,"[(AAPL, 14), (ABC, 14), (ABMD, 4), (AMCR, 109)...",0.519847,-0.021928
25,2013-03-21,2015-03-23,"[(AAPL, 12), (ABC, 14), (ABMD, 4), (AMCR, 109)...",0.518836,-0.024039
26,2013-03-22,2015-03-24,"[(AAPL, 12), (ABC, 15), (ABMD, 4), (AMCR, 112)...",0.515237,-0.024244
27,2013-03-25,2015-03-25,"[(AAPL, 10), (ABC, 14), (ABMD, 5), (AMCR, 115)...",0.517862,-0.027472
28,2013-03-26,2015-03-26,"[(AAPL, 10), (ABC, 14), (ABMD, 6), (AMCR, 120)...",0.502388,-0.018706
29,2013-03-27,2015-03-27,"[(AAPL, 13), (ABC, 15), (ABMD, 5), (AMCR, 116)...",0.492495,-0.017607
30,2013-03-28,2015-03-30,"[(AAPL, 15), (ABC, 15), (ABMD, 5), (AMCR, 112)...",0.498548,-0.020097
31,2013-04-03,2015-04-02,"[(AAPL, 15), (ABC, 13), (ABMD, 6), (AMCR, 102)...",0.513791,-0.020025


In [485]:
result.describe()

,expected_return,actual_return
count,1093.000000,1088.000000
mean,0.447638,0.225478
std,0.042658,0.169545
min,0.338114,-0.114493
25%,0.419987,0.113510
50%,0.446649,0.208761
75%,0.474406,0.317831
max,0.573334,0.803709


In [487]:
result.to_csv('result_500_max_sharpe.csv',index=False)

In [484]:
result

,start,end,portfolio,expected_return,actual_return
0,2013-01-02,2015-01-02,"[(ABC, 10), (ABMD, 1), (AMCR, 124), (BBY, 4), ...",0.534718,0.114611
1,2013-01-03,2015-01-05,"[(ABC, 10), (ABMD, 1), (AMCR, 123), (BBY, 3), ...",0.523858,0.123409
2,2013-01-04,2015-01-06,"[(ABC, 11), (ABMD, 2), (AMCR, 125), (BBY, 3), ...",0.514343,0.149813
3,2013-01-07,2015-01-07,"[(ABC, 11), (ABMD, 2), (AMCR, 112), (BBY, 3), ...",0.506118,0.145699
4,2013-01-08,2015-01-08,"[(ABC, 10), (ABMD, 2), (AMCR, 108), (AMD, 1), ...",0.512644,0.130540
5,2013-01-09,2015-01-09,"[(ABC, 10), (ABMD, 2), (AMCR, 107), (BBY, 3), ...",0.523753,0.119110
6,2013-01-10,2015-01-12,"[(ABC, 12), (ABMD, 3), (AMCR, 110), (BBY, 2), ...",0.513385,0.136310
7,2013-01-11,2015-01-13,"[(ABC, 10), (ABMD, 5), (AMCR, 113), (CBOE, 1),...",0.508808,0.121721
8,2013-01-14,2015-01-14,"[(ABC, 10), (ABMD, 4), (AMCR, 115), (AMD, 5), ...",0.506996,0.120682
9,2013-01-16,2015-01-16,"[(ABC, 9), (ABMD, 3), (AMCR, 100), (CNC, 5), (...",0.484019,NaN
